In [1]:
import json
import math
import os
import numpy as np
import pandas as pd
import requests
import browser_cookie3
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
%matplotlib inline

In [2]:
if not os.path.exists('input16'):
    cookies = browser_cookie3.firefox(domain_name="adventofcode.com")
    session_cookie = [c for c in list(cookies) if c.name == "session"][0].value
    resp = requests.get('https://adventofcode.com/2021/day/16/input', cookies={'session': session_cookie})
    with open('input16', 'w') as f:
        f.write(resp.text)

In [57]:
with open('input16', 'r') as f:
    lines = [l.strip() for l in f.readlines()]
line = '1' + lines[0]

## Part 1

In [ ]:
line = '1' + 'A0016C880162017C3686B18A3D4780'

In [ ]:
binary = bin(int(line, 16))[3:]

In [ ]:
def parse_packets(binary):
    packets = []
    i = 0
    if sum(int(v) for v in binary) == 0:
        return [], 0
    pversion = int(binary[i:i+3], 2)
    ptype = int(binary[i+3:i+6], 2)
    i += 6
    if ptype == 4:
        # Literal
        value_groups = []
        while True:
            value_groups.append(binary[i+1:i+5])
            i += 5
            if int(binary[i-5]) == 0:
                break
        value = int(''.join(value_groups), 2)
        packets.append((pversion, ptype, value))
    else:
        # Operator
        length_type_id = int(binary[i])
        i += 1
        if length_type_id == 0:
            # Length of subpackets specified
            sub_packets_length = int(binary[i:i+15], 2)
            i += 15
            end_subpackets_index = i + sub_packets_length
            while i < end_subpackets_index:
                new_packets, num_bits = parse_packets(binary[i:])
                packets.extend(new_packets)
                i += num_bits
            packets.append((pversion, ptype, length_type_id, sub_packets_length))
        else:
            # Number of subpackets specified
            num_sub_packets = int(binary[i:i+11], 2)
            i += 11
            num_new_packets = 0
            while num_new_packets < num_sub_packets:
                new_packets, num_bits = parse_packets(binary[i:])
                packets.extend(new_packets)
                num_new_packets += 1
                i += num_bits
            packets.append((pversion, ptype, length_type_id, num_sub_packets))
    return packets, i

In [ ]:
packets, _ = parse_packets(binary)
sum([p[0] for p in packets])

## Part 2

In [51]:
line = '1' + '9C0141080250320F1802104A08'

In [58]:
binary = bin(int(line, 16))[3:]

In [59]:
def parse_packets(binary):
    packets = []
    i = 0
    if sum(int(v) for v in binary) == 0:
        return [], 0
    pversion = int(binary[i:i+3], 2)
    ptype = int(binary[i+3:i+6], 2)
    i += 6
    if ptype == 4:
        # Literal
        value_groups = []
        while True:
            value_groups.append(binary[i+1:i+5])
            i += 5
            if int(binary[i-5]) == 0:
                break
        value = int(''.join(value_groups), 2)
        packets.append((ptype, value))
    else:
        # Operator
        length_type_id = int(binary[i])
        i += 1
        if length_type_id == 0:
            # Length of subpackets specified
            sub_packets_length = int(binary[i:i+15], 2)
            i += 15
            end_subpackets_index = i + sub_packets_length
            sub_packets = []
            while i < end_subpackets_index:
                new_packets, num_bits = parse_packets(binary[i:])
                sub_packets.extend(new_packets)
                i += num_bits
            packets.append((ptype, sub_packets))
        else:
            # Number of subpackets specified
            num_sub_packets = int(binary[i:i+11], 2)
            i += 11
            sub_packets = []
            while len(sub_packets) < num_sub_packets:
                new_packets, num_bits = parse_packets(binary[i:])
                sub_packets.extend(new_packets)
                i += num_bits
            packets.append((ptype, sub_packets))
    return packets, i

In [60]:
packets, _ = parse_packets(binary)

In [61]:
def get_packets_value(packets):
    values = []
    for packet in packets:
        ptype = packet[0]
        if ptype == 4:
            values.append(packet[1])
        else:
            sub_packets = packet[1]
            sub_packets_value = get_packets_value(sub_packets)
            if ptype == 0:
                values.append(sum(sub_packets_value))
            elif ptype == 1:
                values.append(math.prod(sub_packets_value))
            elif ptype == 2:
                values.append(min(sub_packets_value))
            elif ptype == 3:
                values.append(max(sub_packets_value))
            elif ptype == 5:
                values.append(int(sub_packets_value[0] > sub_packets_value[1]))
            elif ptype == 6:
                values.append(int(sub_packets_value[0] < sub_packets_value[1]))
            elif ptype == 7:
                values.append(int(sub_packets_value[0] == sub_packets_value[1]))
    return values

In [63]:
get_packets_value(packets)[0]

2223947372407